# Anggota Kelompok
1. Andreandhiki Riyanta Putra (23/517511/PA/22191)
2. Andrian Danar Perdana (23/513040/PA/21917)
3. Daffa Indra Wibowo (23/518514/PA/22253)
4. Muhammad Argya Vityasy (23/522547/PA/22475)

# Problem 1: Hash Function for Integrity Check

This notebook demonstrates how to use a cryptographic hash function (SHA-256) to check file integrity. We will:
1.  Implement a function to compute the SHA-256 hash of a file.
2.  Store this hash.
3.  Implement a function to verify the file's integrity by re-computing its hash and comparing it with the stored one.
4.  Discuss the choice of SHA-256 and the risks of using weaker hash functions.
5.  Demonstrate the process with an example.

We'll use Python's built-in `hashlib` module.

In [ ]:
# Import necessary libraries
import hashlib
import os

# Define the block size for reading the file (to handle large files efficiently)
BLOCK_SIZE = 65536 # The size of each read from the file

def compute_sha256_hash(file_path):
    """
    Computes the SHA-256 hash of a given file.

    Args:
        file_path (str): The path to the file.

    Returns:
        str: The hexadecimal representation of the SHA-256 hash,
             or None if the file cannot be read.
    """
    sha256_hash = hashlib.sha256()
    try:
        with open(file_path, "rb") as f:
            # Read the file in chunks
            for byte_block in iter(lambda: f.read(BLOCK_SIZE), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()
    except IOError:
        print(f"Error: Could not read file at {file_path}")
        return None

def store_hash(hash_value, storage_path="stored_hash.txt"):
    """
    Stores the hash value to a file.

    Args:
        hash_value (str): The hash to store.
        storage_path (str): The file path to store the hash.
    """
    try:
        with open(storage_path, "w") as f:
            f.write(hash_value)
        print(f"Hash stored successfully in {storage_path}")
    except IOError:
        print(f"Error: Could not write hash to {storage_path}")

def retrieve_stored_hash(storage_path="stored_hash.txt"):
    """
    Retrieves the stored hash value from a file.

    Args:
        storage_path (str): The file path from where to retrieve the hash.

    Returns:
        str: The stored hash, or None if the file cannot be read.
    """
    try:
        with open(storage_path, "r") as f:
            return f.read().strip()
    except IOError:
        print(f"Error: Could not read stored hash from {storage_path}")
        return None

def verify_integrity(file_path, stored_hash_value):
    """
    Verifies the integrity of a file by comparing its current hash
    with a previously stored hash.

    Args:
        file_path (str): The path to the file to verify.
        stored_hash_value (str): The previously computed and stored hash.

    Returns:
        bool: True if the integrity is verified (hashes match), False otherwise.
    """
    if stored_hash_value is None:
        print("Error: No stored hash provided for verification.")
        return False

    current_hash = compute_sha256_hash(file_path)
    if current_hash is None:
        print("Error: Could not compute current hash for verification.")
        return False

    print(f"\nStored Hash:   {stored_hash_value}")
    print(f"Current Hash:  {current_hash}")

    if current_hash == stored_hash_value:
        print("File integrity VERIFIED. The file has not been modified.")
        return True
    else:
        print("File integrity COMPROMISED! The file has been modified.")
        return False

## Demonstration

Let's create a sample file, compute its hash, store it, and then verify its integrity. We will then modify the file and see how the verification fails.


In [ ]:
# 1. Create a dummy file
sample_file_path = "sample_document.txt"
original_content = "This is a secret document. Handle with care.\nLine 2 for more content."
with open(sample_file_path, "w") as f:
    f.write(original_content)
print(f"Created sample file: {sample_file_path}")

# 2. Compute and store its SHA-256 hash
initial_hash = compute_sha256_hash(sample_file_path)
if initial_hash:
    hash_storage_file = "sample_document_hash.txt"
    store_hash(initial_hash, hash_storage_file)

    # 3. Retrieve the stored hash (as if done later)
    retrieved_hash = retrieve_stored_hash(hash_storage_file)

    # 4. Verify the integrity of the original file
    print("\n--- Verifying original file ---")
    verify_integrity(sample_file_path, retrieved_hash)

    # 5. Modify the file slightly
    print("\n--- Modifying the file ---")
    with open(sample_file_path, "a") as f: # Append some text
        f.write("\nThis line was added later.")
    print(f"Content of '{sample_file_path}' has been modified.")

    # 6. Verify the integrity of the modified file
    print("\n--- Verifying modified file ---")
    verify_integrity(sample_file_path, retrieved_hash) # Use the original stored hash

    # Clean up dummy files (optional)
    # os.remove(sample_file_path)
    # os.remove(hash_storage_file)
else:
    print("Could not compute initial hash. Demonstration aborted.")

Created sample file: sample_document.txt
Hash stored successfully in sample_document_hash.txt

--- Verifying original file ---

Stored Hash:   7c9bd2f9b3c37f49aa740d1664d2aa04c920d687d5aa4a9df409268ce3384098
Current Hash:  7c9bd2f9b3c37f49aa740d1664d2aa04c920d687d5aa4a9df409268ce3384098
File integrity VERIFIED. The file has not been modified.

--- Modifying the file ---
Content of 'sample_document.txt' has been modified.

--- Verifying modified file ---

Stored Hash:   7c9bd2f9b3c37f49aa740d1664d2aa04c920d687d5aa4a9df409268ce3384098
Current Hash:  0883762d4f25eda0d25eee11d4cbd093010be81ccf756d66b4bd37fed82b7190
File integrity COMPROMISED! The file has been modified.


## Demonstration of Integrity Verification by Altering the File

As seen in the Python code execution above:
1.  We created `sample_document.txt` and calculated its SHA-256 hash. This hash was stored.
2.  Verification against the original file and the stored hash showed: "File integrity VERIFIED."
3.  We then altered `sample_document.txt` by appending a new line.
4.  When we re-verified the *modified* file against the *original stored hash*, the newly computed hash was different. The output correctly showed: "File integrity COMPROMISED! The file has been modified." This demonstrates that even a slight alteration to the file results in a different hash value, allowing us to detect tampering.


## Explanation

### Why SHA-256 was chosen:

SHA-256 (Secure Hash Algorithm 256-bit) was chosen for several reasons:

1.  **Security Strength:** SHA-256 is a member of the SHA-2 family of hash functions, which are currently considered secure against known cryptanalytic attacks. It produces a 256-bit hash value, making it highly resistant to collision attacks (finding two different inputs that produce the same hash) and preimage attacks (finding an input that produces a given hash).
2.  **Standardization and Wide Adoption:** SHA-256 is a well-established standard (FIPS PUB 180-4) and is widely used in various security applications and protocols, including TLS/SSL, digital signatures, and blockchain technology. This widespread adoption implies significant scrutiny and trust from the cryptographic community.
3.  **Avalanche Effect:** A good cryptographic hash function exhibits a strong avalanche effect, meaning a small change in the input data (e.g., flipping a single bit) results in a drastically different hash value. SHA-256 demonstrates this property effectively, which is crucial for integrity checking as even minor modifications are easily detectable.
4.  **Performance:** While more computationally intensive than older algorithms like MD5, SHA-256 offers a good balance between security and performance for most file integrity applications. Modern hardware often includes acceleration for SHA-2 operations.

### What might happen if a weaker hash (e.g., MD5) were used:

MD5 (Message Digest 5) is an older hash function that produces a 128-bit hash value. It is now considered cryptographically broken and unsuitable for security-critical applications like file integrity verification due to several vulnerabilities:

1.  **Collision Vulnerability:** The most significant issue with MD5 is that practical collision attacks have been demonstrated. This means an attacker can intentionally create two different files (e.g., a legitimate file and a malicious one) that produce the *same* MD5 hash. If MD5 were used for integrity checking, an attacker could replace a legitimate file with a malicious one that has the same MD5 hash, and the integrity check would falsely report that the file is unchanged.
2.  **Preimage Attack Susceptibility (Theoretical):** While full preimage attacks are still computationally very difficult for MD5, its smaller hash size makes it theoretically more vulnerable than SHA-256.
3.  **Lack of Trust:** Due to its known vulnerabilities, using MD5 for security purposes erodes trust in the system. Regulatory bodies and security standards often mandate the use of stronger hash functions like those in the SHA-2 family.

In the context of our file integrity checker:
* If MD5 were used, an attacker could modify `sample_document.txt` to include malicious content while crafting the modification in such a way that the MD5 hash remains the same as the original. The integrity check would then fail to detect the tampering.
* The probability of accidental collisions (two different benign files having the same hash) is also higher with MD5's 128-bit output compared to SHA-256's 256-bit output, though still very low for random data. The primary concern is intentional, malicious collisions.

Therefore, using SHA-256 provides a much higher assurance of data integrity.

# Problem 2: Message Authentication using HMAC

This notebook demonstrates how to use a Hash-based Message Authentication Code (HMAC) to authenticate messages. We will:
1.  Implement functions to generate an HMAC using SHA-256 (custom implementation based on the HMAC algorithm).
2.  Implement a function to verify a received message and its HMAC.
3.  Simulate scenarios where an attacker modifies the message or the HMAC.
4.  Discuss how HMAC differs from plain hashing, its resistance to forgery, and the importance of the shared secret key.

We will use Python's built-in `hashlib` module for the underlying hash function (SHA-256).

In [ ]:
# Import necessary libraries
import hashlib
import hmac # For comparison and secure digest comparison

# SHA-256 block size is 64 bytes (512 bits)
# This is inherent to SHA-256 and used in the HMAC construction.
HMAC_BLOCK_SIZE = 64
# Inner and outer padding constants for HMAC (as per RFC 2104)
IPAD_BYTE = 0x36
OPAD_BYTE = 0x5C

def prepare_hmac_key(secret_key_bytes, hash_algo=hashlib.sha256):
    """
    Prepares the secret key for HMAC according to RFC 2104.
    - If key is longer than block size, it's hashed using the specified hash_algo.
    - If key is shorter than block size, it's padded with zeros to match block size.

    Args:
        secret_key_bytes (bytes): The secret key.
        hash_algo (function): The hash function to use (e.g., hashlib.sha256).
                              The block size is assumed to be HMAC_BLOCK_SIZE.

    Returns:
        bytes: The processed key (K_0 in HMAC specification), sized to HMAC_BLOCK_SIZE.
    """
    key = secret_key_bytes

    # If key is longer than block size, hash it
    if len(key) > HMAC_BLOCK_SIZE:
        hasher = hash_algo()
        hasher.update(key)
        key = hasher.digest()

    # If key is shorter than block size, pad with zeros to the right
    if len(key) < HMAC_BLOCK_SIZE:
        # The padding consists of appending NUL bytes
        key = key + b'\x00' * (HMAC_BLOCK_SIZE - len(key))

    return key

def generate_hmac_sha256_custom(secret_key_str, message_str):
    """
    Generates an HMAC-SHA256 for a given message and secret key using a custom implementation.
    The formula is: HMAC(K, m) = H((K_0 XOR opad) || H((K_0 XOR ipad) || m))

    Args:
        secret_key_str (str): The shared secret key (will be encoded to UTF-8 bytes).
        message_str (str): The message to authenticate (will be encoded to UTF-8 bytes).

    Returns:
        str: The hexadecimal representation of the HMAC-SHA256.
    """
    secret_key_bytes = secret_key_str.encode('utf-8')
    message_bytes = message_str.encode('utf-8')

    # 1. Prepare the key (K_0)
    # Ensure K_0 is BLOCK_SIZE bytes long (hash if longer, pad if shorter)
    k_0 = prepare_hmac_key(secret_key_bytes, hashlib.sha256)

    # 2. Create K_0 XOR ipad and K_0 XOR opad
    #    ipad is 0x36 repeated BLOCK_SIZE times
    #    opad is 0x5C repeated BLOCK_SIZE times

    k_ipad_bytes = bytearray(HMAC_BLOCK_SIZE)
    k_opad_bytes = bytearray(HMAC_BLOCK_SIZE)

    for i in range(HMAC_BLOCK_SIZE):
        k_ipad_bytes[i] = k_0[i] ^ IPAD_BYTE
        k_opad_bytes[i] = k_0[i] ^ OPAD_BYTE

    # 3. Inner hash: H((K_0 XOR ipad) || message)
    inner_hasher = hashlib.sha256()
    inner_hasher.update(bytes(k_ipad_bytes)) # k_ipad_bytes is already bytes-like
    inner_hasher.update(message_bytes)
    inner_hash_result = inner_hasher.digest() # Result is in bytes

    # 4. Outer hash: H((K_0 XOR opad) || inner_hash_result)
    outer_hasher = hashlib.sha256()
    outer_hasher.update(bytes(k_opad_bytes)) # k_opad_bytes is already bytes-like
    outer_hasher.update(inner_hash_result)

    return outer_hasher.hexdigest() # Return as a hex string

def verify_hmac_sha256_custom(secret_key_str, message_str, received_mac_hex):
    """
    Verifies a received message and HMAC (hex string) against a re-computed HMAC
    using the custom implementation.

    Args:
        secret_key_str (str): The shared secret key.
        message_str (str): The received message.
        received_mac_hex (str): The received HMAC (hexadecimal string).

    Returns:
        bool: True if the MAC is valid, False otherwise.
    """
    # Re-generate the HMAC for the received message using the shared secret key
    computed_mac_hex = generate_hmac_sha256_custom(secret_key_str, message_str)

    print(f"\nReceived MAC (Custom Verify):   {received_mac_hex}")
    print(f"Computed MAC (Custom Verify):   {computed_mac_hex}")

    # Securely compare the MACs. Python's hmac.compare_digest is designed for this
    # to prevent timing attacks.
    # For this assignment, using hmac.compare_digest is good practice.
    is_valid = hmac.compare_digest(computed_mac_hex.encode('utf-8'), received_mac_hex.encode('utf-8'))

    if is_valid:
        print("HMAC VERIFIED (Custom). Message is authentic and integrity is intact.")
        return True
    else:
        print("HMAC VERIFICATION FAILED (Custom)! Message may have been tampered with or key is incorrect.")
        return False

# For comparison and to use hmac.compare_digest, let's also show Python's built-in hmac module
def generate_hmac_with_module(secret_key_str, message_str):
    """Generates HMAC using Python's hmac module."""
    key_bytes = secret_key_str.encode('utf-8')
    message_bytes = message_str.encode('utf-8')
    mac_object = hmac.new(key_bytes, message_bytes, hashlib.sha256)
    return mac_object.hexdigest()

def verify_hmac_with_module(secret_key_str, message_str, received_mac_hex):
    """Verifies HMAC using Python's hmac module (and its compare_digest)."""
    key_bytes = secret_key_str.encode('utf-8')
    message_bytes = message_str.encode('utf-8')

    # Generate expected MAC
    expected_mac_object = hmac.new(key_bytes, message_bytes, hashlib.sha256)
    expected_mac_hex = expected_mac_object.hexdigest()

    print(f"\nReceived MAC (Module Verify):   {received_mac_hex}")
    print(f"Computed MAC (Module Verify):   {expected_mac_hex}")

    is_valid = hmac.compare_digest(expected_mac_hex.encode('utf-8'), received_mac_hex.encode('utf-8'))

    if is_valid:
        print("HMAC VERIFIED (Module). Message is authentic and integrity is intact.")
        return True
    else:
        print("HMAC VERIFICATION FAILED (Module)! Message may have been tampered with or key is incorrect.")
        return False

## Demonstration of HMAC Generation and Verification

Let's define a message and a secret key, then generate and verify an HMAC using both our custom implementation and Python's `hmac` module.


In [ ]:
# Shared secret key (known to sender and receiver)
shared_secret_key = "myVerySecureHMACKey!@#$"

# Original message
original_message = "This is a top secret message for authorized personnel only."

# --- Using our custom HMAC implementation ---
print("--- Using Custom HMAC Implementation ---")
# Sender: Generate HMAC
generated_mac_custom = generate_hmac_sha256_custom(shared_secret_key, original_message)
print(f"Original Message: '{original_message}'")
print(f"Generated HMAC (Custom): {generated_mac_custom}")

# Receiver: Verify HMAC
print("\nReceiver verifying the original message and MAC (Custom):")
is_valid_custom = verify_hmac_sha256_custom(shared_secret_key, original_message, generated_mac_custom)
print(f"Verification Result (Custom): {'Valid' if is_valid_custom else 'Invalid'}")

# --- Using Python's hmac module for comparison ---
print("\n\n--- Using Python's hmac Module (for reference and robust comparison) ---")
generated_mac_module = generate_hmac_with_module(shared_secret_key, original_message)
print(f"Original Message: '{original_message}'")
print(f"Generated HMAC (Module): {generated_mac_module}")

print("\nReceiver verifying the original message and MAC (Module):")
is_valid_module = verify_hmac_with_module(shared_secret_key, original_message, generated_mac_module)
print(f"Verification Result (Module): {'Valid' if is_valid_module else 'Invalid'}")

# --- Final check: Does our custom implementation match the standard module? ---
if generated_mac_custom == generated_mac_module:
    print("\n\nSUCCESS: Custom HMAC implementation output matches Python's hmac module output.")
else:
    print("\n\nWARNING: Custom HMAC implementation output does NOT match Python's hmac module output. Check implementation details.")



--- Using Custom HMAC Implementation ---
Original Message: 'This is a top secret message for authorized personnel only.'
Generated HMAC (Custom): 9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff

Receiver verifying the original message and MAC (Custom):

Received MAC (Custom Verify):   9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff
Computed MAC (Custom Verify):   9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff
HMAC VERIFIED (Custom). Message is authentic and integrity is intact.
Verification Result (Custom): Valid


--- Using Python's hmac Module (for reference and robust comparison) ---
Original Message: 'This is a top secret message for authorized personnel only.'
Generated HMAC (Module): 9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff

Receiver verifying the original message and MAC (Module):

Received MAC (Module Verify):   9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff
Computed MAC (Module Verify

## Simulating Attack Scenarios

Let's observe how HMAC verification responds when an attacker attempts to tamper with the message or the MAC.


In [ ]:
# We'll use the MAC generated by our custom function for these scenarios
# Assume 'generated_mac_custom' is the MAC accompanying the 'original_message'

# Scenario 1: Attacker modifies the message but sends the original MAC
tampered_message = "This is a FAKE public message for everyone to see." # Attacker changes the message
print("\n--- Scenario 1: Attacker modifies the message ---")
print(f"Original Message: '{original_message}'")
print(f"Tampered Message: '{tampered_message}'")
print(f"Original HMAC:    '{generated_mac_custom}' (attacker sends this MAC with the tampered message)")

print("\nReceiver verifying the tampered message with the original MAC (using custom verify):")
is_valid_tampered_msg = verify_hmac_sha256_custom(shared_secret_key, tampered_message, generated_mac_custom)
print(f"Verification Result: {'Valid' if is_valid_tampered_msg else 'Invalid'}")
# Expected: Invalid. The MAC was for the original message, not the tampered one.

# Scenario 2: Attacker keeps the original message but modifies the MAC
# (This is hard for an attacker to do meaningfully without the key, they'd just be guessing a MAC)
original_message_for_scenario2 = "Another secure dispatch."
mac_for_scenario2_custom = generate_hmac_sha256_custom(shared_secret_key, original_message_for_scenario2)

# Attacker slightly changes the MAC (e.g., flips a bit or changes a character)
if len(mac_for_scenario2_custom) > 0:
    # Change the last character of the MAC to something different
    last_char = mac_for_scenario2_custom[-1]
    new_last_char = 'a' if last_char != 'a' else 'b' # Ensure it's different
    tampered_mac = mac_for_scenario2_custom[:-1] + new_last_char
else:
    tampered_mac = "invalidtamperedmac123" # Fallback if MAC was empty

print("\n\n--- Scenario 2: Attacker modifies the MAC ---")
print(f"Original Message: '{original_message_for_scenario2}'")
print(f"Original MAC:     '{mac_for_scenario2_custom}'")
print(f"Tampered MAC:     '{tampered_mac}' (attacker sends this with the original message)")

print("\nReceiver verifying the original message with the tampered MAC (using custom verify):")
is_valid_tampered_mac = verify_hmac_sha256_custom(shared_secret_key, original_message_for_scenario2, tampered_mac)
print(f"Verification Result: {'Valid' if is_valid_tampered_mac else 'Invalid'}")
# Expected: Invalid. The MAC has been corrupted and won't match the one computed by the receiver.

# Scenario 3: Attacker fabricates both a message and a MAC without knowing the key
attackers_fabricated_message = "Eve says: Launch the attack at dawn!"
# Attacker doesn't have the shared_secret_key.
# They might try to guess a key, use a different key, or just make up a random string for the MAC.
# Or, more naively, they might compute a plain hash of their message and try to pass it off as an HMAC.
attackers_fabricated_mac_attempt = hashlib.sha256(attackers_fabricated_message.encode('utf-8')).hexdigest() # This is a plain hash, NOT an HMAC

print("\n\n--- Scenario 3: Attacker sends a new message and a fake MAC (e.g., a plain hash) ---")
print(f"Attacker's Message: '{attackers_fabricated_message}'")
print(f"Attacker's Fake MAC (plain hash): '{attackers_fabricated_mac_attempt}'")

print("\nReceiver verifying attacker's message with attacker's fake MAC (using custom verify):")
is_valid_attacker_attempt = verify_hmac_sha256_custom(shared_secret_key, attackers_fabricated_message, attackers_fabricated_mac_attempt)
print(f"Verification Result: {'Valid' if is_valid_attacker_attempt else 'Invalid'}")
# Expected: Invalid. The MAC computed by the receiver using the true shared_secret_key will not match the attacker's fake MAC.



--- Scenario 1: Attacker modifies the message ---
Original Message: 'This is a top secret message for authorized personnel only.'
Tampered Message: 'This is a FAKE public message for everyone to see.'
Original HMAC:    '9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff' (attacker sends this MAC with the tampered message)

Receiver verifying the tampered message with the original MAC (using custom verify):

Received MAC (Custom Verify):   9518f463ac8a3b4fc96ce5d665430162fffba83931c5189862788bdf4ed778ff
Computed MAC (Custom Verify):   22c9e8a0a730c86a9d5198310dfa5361ae936a503af80f1eb01f2189cb743e29
HMAC VERIFICATION FAILED (Custom)! Message may have been tampered with or key is incorrect.
Verification Result: Invalid


--- Scenario 2: Attacker modifies the MAC ---
Original Message: 'Another secure dispatch.'
Original MAC:     'b649231d18ab3dec80129b63639c2bab524bf881c89c18ba1d2072f44331665f'
Tampered MAC:     'b649231d18ab3dec80129b63639c2bab524bf881c89c18ba1d2072f4433166

## Explanation

### How HMAC differs from plain hashing:

1.  **Use of a Secret Key:**
    * **Plain Hashing:** A standard cryptographic hash function (like SHA-256) takes a single input (the message) and produces a fixed-size digest (the hash). The formula is essentially `Hash = H(message)`. Anyone who has access to the message can compute this hash.
    * **HMAC (Hash-based Message Authentication Code):** HMAC incorporates a secret cryptographic key into the hashing process. The general HMAC construction is `HMAC(K, m) = H((K_0 XOR opad) || H((K_0 XOR ipad) || m))`, where `K` is the secret key, `m` is the message, `H` is the chosen hash function, and `ipad` and `opad` are specific padding constants. `K_0` is the key `K` processed to fit the block size of the hash function (hashed if too long, padded if too short). Crucially, only parties possessing the secret key `K` can generate or correctly verify the HMAC.

2.  **Purpose and Guarantees:**
    * **Plain Hashing:** Primarily provides **data integrity**. It can detect if the data has been accidentally or maliciously altered after the hash was computed. However, it does *not* provide **authentication** on its own because anyone can compute the hash of any message. If an attacker changes a message, they can simply recompute the plain hash for the modified message.
    * **HMAC:** Provides both **data integrity** (like a plain hash) AND **message authentication**.
        * **Integrity:** If the message is changed, the HMAC will change.
        * **Authentication:** Because the HMAC calculation depends on the shared secret key, a valid HMAC tag verifies that the message originated from a party that knows this secret key. It confirms the *source* of the message, not just its state.

3.  **Security Against Specific Attacks:**
    * Plain hashes, if used naively for authentication (e.g., `H(secret || message)` or `H(message || secret)`), can be vulnerable to certain attacks like "length extension attacks" (for `H(secret || message)`) or offline hashing of the secret if the hash output is known.
    * The HMAC construction (with its specific nested hashing and use of `ipad` and `opad`) was formally proven to be secure (a PRF - Pseudorandom Function) as long as the underlying hash function `H` meets certain cryptographic properties (e.g., collision resistance). This design specifically thwarts length extension attacks and other common pitfalls of ad-hoc keyed hashing.

### Why HMAC is resistant to forgery (include an example where verification fails due to tampering):

HMAC's resistance to forgery is fundamentally tied to the secrecy of the shared key. An attacker who does not know the key cannot produce a valid HMAC tag for a message of their choosing (or a modified message).

1.  **Inability to Compute a Valid MAC without the Key:**
    * If an attacker intercepts a message `M` and its legitimate HMAC tag `T = HMAC(K, M)`, and they modify `M` to `M'`, they cannot compute the correct new tag `T' = HMAC(K, M')` because they lack `K`.
    * If they simply send `(M', T)` (tampered message, original tag), the receiver will compute `HMAC(K, M')` using their copy of `K`. This computed tag will not match the received tag `T`, and the verification will fail. This is demonstrated in **Scenario 1** in the code above:
        * **Legitimate Sender (Alice):**
            * Message (`M`): "Transfer $100 to Bob."
            * Secret Key (`K`): `shared_secret_key`
            * HMAC (`Tag_Alice`): `generate_hmac_sha256_custom(K, M)`
            * Alice sends (`M`, `Tag_Alice`) to the Bank.
        * **Attacker (Eve):**
            * Intercepts (`M`, `Tag_Alice`).
            * Changes `M` to `M'` ("Transfer $1000 to Eve.").
            * Eve does *not* know `K`. She cannot compute `Tag_Eve_Correct = generate_hmac_sha256_custom(K, M')`.
            * Eve sends (`M'`, `Tag_Alice`) to the Bank (i.e., tampered message with the original, now incorrect, MAC).
        * **Receiver (Bank):**
            * Receives (`M'`, `Tag_Alice`).
            * The Bank knows `K`.
            * Bank computes `Tag_Bank_Computed = generate_hmac_sha256_custom(K, M')`.
            * Bank compares `Tag_Bank_Computed` with the received `Tag_Alice`. Since `M'` is different from `M`, `Tag_Bank_Computed` will (with overwhelming probability) be different from `Tag_Alice`.
            * Verification fails. The bank detects tampering and rejects the fraudulent transaction. The output in Scenario 1 shows: "HMAC VERIFICATION FAILED (Custom)! Message may have been tampered with or key is incorrect."

2.  **Difficulty of Guessing/Predicting HMAC Output:**
    * Cryptographic hash functions (like SHA-256) used within HMAC are designed to be one-way (hard to reverse) and to exhibit a strong "avalanche effect" (a small change in input drastically changes the output).
    * This makes it computationally infeasible for an attacker to guess a valid HMAC tag for a given message without knowing the key, or to find a different message that produces the same HMAC tag (a collision related to the key).

3.  **Protection Against Replay (Requires Additional Mechanisms):**
    * It's important to note that HMAC itself authenticates the origin and integrity of a *single* message. It does not inherently protect against "replay attacks," where an attacker resends a valid, previously captured message and its HMAC.
    * To prevent replay attacks, systems using HMAC often incorporate additional mechanisms like sequence numbers, timestamps, or nonces within the message content *before* the HMAC is computed. These make each message unique, so a replayed message would be detected.

### Discuss why using a shared secret key is crucial:

The shared secret key is the absolute linchpin of HMAC's security. Its cruciality can be highlighted as follows:

1.  **Establishes the Basis of Authentication:** The key is the shared secret that forms the trust relationship between the communicating parties. Only entities possessing this key can generate a valid HMAC. If there's no secret, or if the key is public knowledge, the HMAC offers no more authentication than a plain hash – anyone could generate it.
2.  **Enables Source Verification:** A valid HMAC tag assures the receiver that the message came from someone who knows the key. This is the "authentication" part of "Message Authentication Code." Without the key, this assurance vanishes.
3.  **Provides the "MAC" in HMAC:** The "MA" in HMAC stands for Message Authentication. This authentication is entirely predicated on the key being secret and shared only among authorized parties.
4.  **Prevents Forgery:** As detailed above, the inability of an attacker (who doesn't know the key) to compute a valid HMAC for an arbitrary or modified message is the primary defense against forgery. The secrecy of the key is what makes this defense effective.
5.  **Integrity Becomes Authenticated Integrity:** While a plain hash provides integrity, HMAC provides *authenticated integrity*. This means the receiver not only knows the message hasn't changed since the HMAC was computed but also trusts that it was computed by a legitimate party (one knowing the key).

**Consequences of a Compromised Shared Secret Key:**
If the shared secret key is compromised (i.e., an unauthorized party learns it):
* **Complete Loss of Authentication:** The attacker can now generate valid HMACs for any fraudulent messages they create. These messages will appear legitimate to the receiver.
* **Undetectable Tampering:** The attacker can intercept legitimate messages, modify them, and then re-calculate and attach a new, valid HMAC using the compromised key. This tampering would be undetectable by the HMAC verification process alone.
* **Impersonation:** The attacker can impersonate any of the legitimate parties who share that key.
* The security provided by HMAC for that specific key is entirely nullified. All past and future messages secured with that compromised key are suspect.

Therefore, the security of the HMAC system is critically dependent on the security of the shared secret key. Robust key management practices are essential:
* **Secure Generation:** Keys should be generated using a cryptographically secure random number generator.
* **Secure Distribution:** Keys must be shared between parties over a secure channel.
* **Secure Storage:** Keys must be protected from unauthorized access, both at rest and in memory.
* **Limited Scope/Lifetime:** Keys should ideally be used for a limited purpose or time, and rotated periodically.